In [11]:
import os
import zstandard
import json
import sys
import pandas as pd
import psutil

subreddit_list = ['politics', 'memes', 'teenagers', 'AmItheAsshole', 'AskReddit']
comment_fields_list = ['link_id', 'id', 'parent_id', 'author', 'author_fullname', 'permalink',
                       'body', 'total_awards_received', 'score', 'subreddit', 'removal_reason']
submission_fields_list = ['id', 'num_comments', 'author', 'author_fullname', 'permalink', 
                          'selftext', 'total_awards_received', 'score', 'subreddit', 'removal_reason']
data_path = 'data'

In [12]:
rs_list = []
for f in os.listdir(data_path):
    if (f.find('RS')>=0) & (f.find('csv')>=0):
        rs_list.append(os.path.join(data_path, f))
rs_list

['data/RS_2019-03.csv.gz',
 'data/RS_2019-06.csv.gz',
 'data/RS_2019-04.csv.gz',
 'data/RS_2019-01.csv.gz',
 'data/RS_2019-05.csv.gz',
 'data/RS_2019-09.csv.gz',
 'data/RS_2019-12.csv.gz',
 'data/RS_2019-08.csv.gz',
 'data/RS_2018-11.csv.gz',
 'data/RS_2019-11.csv.gz',
 'data/RS_2019-10.csv.gz',
 'data/RS_2019-07.csv.gz',
 'data/RS_2019-02.csv.gz',
 'data/RS_2018-12.csv.gz']

In [13]:
%%time
#fl_path = os.path.join(data_path, 'RS_2019-12.csv.gz')
df_s = pd.concat([pd.read_csv(i, compression = 'gzip', sep = ';') for i in rs_list]
                ).reset_index().drop('index', axis = 1)

CPU times: user 12.1 s, sys: 786 ms, total: 12.9 s
Wall time: 12.9 s


In [14]:
df_s.shape

(3935392, 11)

In [22]:
df_s['text'] = df_s.selftext
df_s.loc[(df_s.selftext.isnull()) | (df_s.selftext.isin(['[deleted]', '[removed]'])), 'text'
        ] = df_s.loc[(df_s.selftext.isnull()) | (df_s.selftext.isin(['[deleted]', '[removed]'])), 'title']

In [24]:
df_s = df_s[df_s.text.notnull()]
df_s = df_s[~df_s.text.isin(['[deleted]', '[removed]'])]

In [25]:
df_s.shape

(3935390, 12)

In [26]:
df_s.tail(2)

,id,num_comments,title,author,author_fullname,permalink,selftext,total_awards_received,score,subreddit,removal_reason,text
3935390,abcubk,1,What is the best new song you heard in 2018?,A_Tricky_one,t2_sbdyn,/r/AskReddit/comments/abcubk/what_is_the_best_...,[removed],NaN,1,AskReddit,NaN,What is the best new song you heard in 2018?
3935391,abcuf1,3,What's your story when you thought you were go...,ronimal48,t2_khnlj,/r/AskReddit/comments/abcuf1/whats_your_story_...,NaN,NaN,3,AskReddit,NaN,What's your story when you thought you were go...


In [27]:
rc_list = []
for f in os.listdir(data_path):
    if (f.find('RC')>=0) & (f.find('csv')>=0):
        rc_list.append(os.path.join(data_path, f))
rc_list

['data/RC_2019-04.csv.gz',
 'data/RC_2019-10.csv.gz',
 'data/RC_2018-12.csv.gz',
 'data/RC_2019-09.csv.gz',
 'data/RC_2018-10.csv.gz',
 'data/RC_2019-05.csv.gz',
 'data/RC_2018-11.csv.gz',
 'data/RC_2019-11.csv.gz',
 'data/RC_2019-06.csv.gz',
 'data/RC_2019-08.csv.gz',
 'data/RC_2019-12.csv.gz',
 'data/RC_2019-01.csv.gz',
 'data/RC_2019-02.csv.gz',
 'data/RC_2019-07.csv.gz',
 'data/RC_2019-03.csv.gz']

In [28]:
%%time
df_c = pd.concat([pd.read_csv(i, compression = 'gzip', sep = ';') for i in rc_list]
                ).reset_index().drop('index', axis = 1)

/home/dima/anaconda3/lib/python3.8/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)
/home/dima/anaconda3/lib/python3.8/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (0,1,2,3,4,5,6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


CPU times: user 3min 49s, sys: 25 s, total: 4min 14s
Wall time: 4min 15s


In [29]:
df_c.shape

(94181545, 13)

In [30]:
df_c.tail(2)

,id,num_comments,author,author_fullname,permalink,selftext,total_awards_received,score,subreddit,removal_reason,link_id,parent_id,body
94181543,ejualmg,NaN,chiplay99,t2_3g6x7l2l,/r/AskReddit/comments/b7ulkq/how_often_to_you_...,None,NaN,3.0,AskReddit,NaN,NaN,NaN,NaN
94181544,ejualmq,NaN,gocamping,t2_1cltf29,/r/AskReddit/comments/b7rka1/what_gets_better_...,None,NaN,2.0,AskReddit,NaN,NaN,NaN,NaN


In [31]:
df_c = df_c[df_c.author != '[deleted]']
df_c = df_c[df_c.link_id.apply(lambda x: len(str(x).split('_')) > 1)]
df_c['subm_id'] = df_c.link_id.apply(lambda x: str(x).split('_')[1])
df_c = df_c[df_c.body.notnull()]

In [39]:
df_c.shape

(16351059, 14)

In [33]:
df_c.tail(2)

,id,num_comments,author,author_fullname,permalink,selftext,total_awards_received,score,subreddit,removal_reason,link_id,parent_id,body,subm_id
42057357,eatm37x,NaN,crankywithakeyboard,t2_i3hlf,/r/AskReddit/comments/a1xsf8/whats_your_christ...,NaN,NaN,1.0,AskReddit,NaN,t3_a1xsf8,t3_a1xsf8,Big families draw names and only buy for that ...,a1xsf8
42057358,eatm385,NaN,Jarkny,t2_2imewmqg,/r/AskReddit/comments/a1xsxm/whats_the_worst_a...,NaN,NaN,2.0,AskReddit,NaN,t3_a1xsxm,t1_eatlust,I know. I’ve always thought how lucky I am tbh.,a1xsxm


In [41]:
%%time
df = pd.merge(df_c[['link_id', 'id', 'author', 'total_awards_received', 'body', 'permalink',
                    'score', 'subreddit', 'subm_id']
                  ].rename(columns = {'body': 'text'}),#.head(20), 
              df_s[['author', 'text', 'id',  'total_awards_received', 'score', 'num_comments']
                  ],
              how = 'inner', 
              left_on = 'subm_id', 
              right_on = 'id',
              suffixes = ['_comment', '_submission']
             ).drop('subm_id', axis = 1)

CPU times: user 18.5 s, sys: 2.06 s, total: 20.6 s
Wall time: 20.8 s


In [42]:
df.head(2).T

,0,1
link_id,t3_a1x406,t3_a1x406
id_comment,eatm38u,eatm4v8
author_comment,DomIstKrieg,dorydoop
total_awards_received_comment,NaN,NaN
text_comment,Same. I would have missed some cool stuff.,Wow. That incredible. Did you get to know the ...
permalink,/r/AskReddit/comments/a1x406/redditors_who_hav...,/r/AskReddit/comments/a1x406/redditors_who_hav...
score_comment,2,2
subreddit,AskReddit,AskReddit
author_submission,dorydoop,dorydoop
text_submission,"Redditors who have come close to suicide, what...","Redditors who have come close to suicide, what..."


In [43]:
df.shape

(9423894, 14)

In [44]:
df.subreddit.value_counts()

AskReddit    9407867
Name: subreddit, dtype: int64

AmItheAsshole, teenagers, politics

In [1]:
%%time
df.to_csv(os.path.join(data_path, 'joined_data_AskReddit.csv.gz'), 
          compression = 'gzip',
          sep = ';',
          index = False
         )

NameError: name 'df' is not defined